In [1]:
import os
from data_partitioner_jsonl import *
from lisa_tagging import *

In [2]:
file_name = 'corpus.jsonl'
dataset = "blogcorpus"

In [3]:
%time
# input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
# partition_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{dataset}', file_name.split('.')[0]) 
# tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
# output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa-{dataset}'
input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
partition_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{dataset}', file_name.split('.')[0]) 
tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa-{dataset}'


# ensures all directories will exist
for directory in [partition_dir, tag_dir, output_dir]:
    os.makedirs(directory, exist_ok=True)


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.63 µs


## Partition the file into 50 parts

In [ ]:
%%time
partition_file(input_path, partition_dir, chunks=2)

## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server


In [ ]:
%%time
import logging
original_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.ERROR)

tag_partitions(
               input_directory=partition_dir,
               output_directory=tag_dir,
               num_workers=2,
#                default_niceness=0
               )
logging.getLogger().setLevel(original_level)


Using 2 workers for tagging


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'EncT5Tokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'EncT5Tokenizer'.
You are using the default legacy behaviour of the <class 'tokenization_enc_t5.EncT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transf

Tagging file /shared/3/projects/hiatus/tagged_data/lisa_partitions/blogcorpus/corpus/partition-1.jsonl with worker 622794
Tagging file /shared/3/projects/hiatus/tagged_data/lisa_partitions/blogcorpus/corpus/partition-2.jsonl with worker 622793
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-2-tagged.jsonl
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-1-tagged.jsonl
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-2-tagged.jsonl
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-2-tagged.jsonl
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-1-tagged.jsonl
Appending chunk to /shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-blogcorpus/corpus/partition-2-tagged.jsonl
Appending chunk to

## Join the partitioned files

In [ ]:
# join_tagged_files(input_directory=tag_dir,
#                   output_file=os.path.join(output_dir, file_name))

## Delete the temp directories

In [ ]:
# delete_partitioned_files(partition_dir)
# delete_partitioned_files(tag_dir)